In [1]:
!git clone https://github.com/karimcossentini/3D_object_reconstruction.git

Cloning into '3D_object_reconstruction'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 100 (delta 8), reused 97 (delta 8), pack-reused 0
Receiving objects: 100% (100/100), 143.24 KiB | 7.54 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
%cd /content/3D_object_reconstruction
!git checkout dev_branch
%cd ..

/content/3D_object_reconstruction
Branch 'dev_branch' set up to track remote branch 'dev_branch' from 'origin'.
Switched to a new branch 'dev_branch'


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Link to pretrained model

# https://drive.google.com/file/d/1fqmLIoNqqY3s-ratcQ-_Jzyi4cZnx5ZW/view?usp=sharing
!cp /content/drive/MyDrive/mvs_best.pth /content/3D_object_reconstruction/MVSFormer/pretrained_weights

In [6]:
%cd /content/3D_object_reconstruction
!pip install pycolmap
!git clone --recursive https://github.com/colmap/pycolmap.git


/content/3D_object_reconstruction
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 78.4 MB/s eta 0:00:00
Cloning into 'pycolmap'...
remote: Enumerating objects: 744, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 744 (delta 261), reused 219 (delta 190), pack-reused 391
Receiving objects: 100% (744/744), 252.27 KiB | 5.87 MiB/s, done.
Resolving deltas: 100% (471/471), done.
Submodule 'pybind11' (https://github.com/pybind/pybind11) registered for path 'pybind11'
Cloning into '/content/3D_object_reconstruction/pycolmap/pybind11'...
remote: Enumerating objects: 26663, done.        
remote: Counting objects: 100% (583/583), done.        
remote: Compressing objects: 100% (226/226), done.        
remote: Total 26663 (delta 366), reused 479 (delta 321), pack-reused 26080        
Receiving objects: 100% (26663/26663), 10.35 MiB | 17.34 MiB/s, done.
Resolving deltas: 100% (18686/18686), done.
Submodule path 'pybin

In [7]:
%cd pycolmap
!pip install .
%cd /content/3D_object_reconstruction

/content/3D_object_reconstruction/pycolmap
Processing /content/3D_object_reconstruction/pycolmap
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycolmap
  Running setup.py clean for pycolmap
Failed to build pycolmap
ERROR: Could not build wheels for pycolmap, which is required to install pyproject.toml-based projects
/content/3D_object_reconstruction


In [8]:
# Custom datasets
# https://drive.google.com/file/d/1OhAncLapePe65JIsY96VNYGQBwAzabeo/view?usp=sharing
# https://drive.google.com/file/d/1QtfhKEK8bGzv-5AFS14af2-0N9P_061R/view?usp=sharing


import zipfile

# Open the ZIP file
with zipfile.ZipFile('/content/drive/MyDrive/images_origin_wallet.zip') as zf:

    # Extract the folder named "example_folder"
    for zip_info in zf.infolist():
        zf.extract(zip_info, path='/content/3D_object_reconstruction')

In [9]:
import pycolmap
import os
# Create a new folder named "my_folder" in the current working directory
folder_name = "/content/3D_object_reconstruction/output"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)

output_path = "/content/3D_object_reconstruction/output"
mvs_path = output_path + "/mvs"
database_path = output_path + "/database.db"
image_dir = "/content/3D_object_reconstruction/images_origin_ordered _49"

pycolmap.extract_features(database_path, image_dir)
pycolmap.match_exhaustive(database_path)
maps = pycolmap.incremental_mapping(database_path, image_dir, output_path)
maps[0].write(output_path)
pycolmap.undistort_images(mvs_path, output_path, image_dir)

In [11]:
reconstruction = pycolmap.Reconstruction("/content/3D_object_reconstruction/output/mvs/sparse/")

folder_name = "/content/3D_object_reconstruction/dense"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
  shutil.rmtree(folder_name)
  os.makedirs(folder_name)
  print('dense deleted')

!cp -r /content/3D_object_reconstruction/output/mvs/images /content/dense

folder_name = "/content/3D_object_reconstruction/dense/sparse"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
  shutil.rmtree(folder_name)
  os.makedirs(folder_name)


reconstruction.write_text("/content/3D_object_reconstruction/dense/sparse")


cp: cannot stat '/content/output/mvs/images': No such file or directory


**Convert the colmap output to mvsnet input**


In [16]:
!python /content/3D_object_reconstruction/colmap2mvsnet.py --dense_folder /content/3D_object_reconstruction/dense


intrinsic[1]
 [[3.17755634e+03 0.00000000e+00 2.03250000e+03]
 [0.00000000e+00 3.17755634e+03 1.51850000e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

extrinsic[1]
 [[ 0.98867494  0.08916087  0.12071537  3.54816306]
 [ 0.02345066  0.70271077 -0.71108905  2.56421395]
 [-0.14822931  0.70586678  0.69266165  2.59307306]
 [ 0.          0.          0.          1.        ]]

/content/3D_object_reconstruction/colmap2mvsnet.py:353: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  zs.append(np.asscalar(transformed[2]))
depth_ranges[1]
 (5.0218053364726405, 0.004353802084687307, 2024.55647651424, 13.83196974240284)

/content/3D_object_reconstruction/colmap2mvsnet.py:395: RuntimeWarning: invalid value encountered in arccos
  theta = (180 / np.pi) * np.arccos(np.dot(cam_center_i - p, cam_center_j - p) / np.linalg.norm(cam_center_i - p) / np.linalg.norm(cam_center_j - p))
view_sel[0]
 [(1, 29.682467402945328), (2, 11.970573229362266), (7, 4.33540535

**Set up MVSFormer configuration**

In [17]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --set python3 /usr/bin/python3.8
!sudo apt-get install python3-pip
!python3 --version


update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in manual mode
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 2,389 kB of archives.
After this operation, 4,933 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python-pip-whl all 20.0.2-5ubuntu1.9 [1,805 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-setuptools all 45.2.0-1ubuntu0.1 [330 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python3-wheel all 0.34.2-1ubuntu0.1 [23.9 kB]
Get:4 http://archive.ubuntu.com/ubuntu

In [10]:
# add this line to /content/3D_object_reconstruction/MVSFormer/requirements.txt right above torch==1.9.0+cu111
# --find-links https://download.pytorch.org/whl/torch_stable.html


In [18]:
!pip install -r /content/3D_object_reconstruction/MVSFormer/requirements.txt


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 15.4 MB 12.6 MB/s 
     |████████████████████████████████| 74 kB 136 kB/s 
     |████████████████████████████████| 60.4 MB 1.1 MB/s 
     |████████████████████████████████| 3.2 MB 68.9 MB/s 
     |████████████████████████████████| 701 kB 73.3 MB/s 
     |████████████████████████████████| 125 kB 75.3 MB/s 
     |████████████████████████████████| 244 kB 69.4 MB/s 
     |████████████████████████████████| 2041.3 MB 2.0 kB/s 
     |████████████████████████████████| 23.2 MB 54.1 MB/s 
     |████████████████████████████████| 74 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 75.4 MB/s 
     |████████████████████████████████| 1.0 MB 58.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=f10824fee7561bf66749532aedf4fbd27281cfc3651eb57ed1587ad4eb20a96c
  Stored in directory: /root/.cache/pip/wheels/c8/d0/a

In [19]:
import shutil

# Specify the source and destination file paths
src_path = '/content/drive/MyDrive/helpers.py'
dst_path = '/usr/local/lib/python3.8/dist-packages/timm/models/layers/helpers.py'

# Copy the source file to the destination file, overwriting it if it already exists
shutil.copyfile(src_path, dst_path)

'/usr/local/lib/python3.8/dist-packages/timm/models/layers/helpers.py'

In [20]:
!git clone https://github.com/YoYo000/fusibile.git

Cloning into 'fusibile'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 85 (delta 29), reused 26 (delta 26), pack-reused 52
Unpacking objects: 100% (85/85), 60.78 KiB | 1.69 MiB/s, done.


In [ ]:
#set -gencode arch=compute_75,code=sm_75 instead of -gencode arch=compute_60,code=sm_60 in fusible/CMakeLists.txt in case you are using Tesla T4 GPU


In [22]:
!pip install cmake

  Using cached cmake-3.26.4-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (24.0 MB)


In [23]:
%cd fusibile
!cmake .
!make
%cd ..

/content/3D_object_reconstruction/fusibile
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.2.0") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
--

In [24]:
import os
import shutil

folder_name = "/content/3D_object_reconstruction/MVSFormer/dtu"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
  shutil.rmtree(folder_name)
  os.makedirs(folder_name)

folder_name = "/content/3D_object_reconstruction/MVSFormer/dtu/scan1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
  shutil.rmtree(folder_name)
  os.makedirs(folder_name)



In [25]:
!cp -r /content/3D_object_reconstruction/dense/cams /content/3D_object_reconstruction/MVSFormer/dtu/scan1
!cp -r /content/3D_object_reconstruction/dense/images /content/3D_object_reconstruction/MVSFormer/dtu/scan1
!cp /content/3D_object_reconstruction/dense/pair.txt /content/3D_object_reconstruction/MVSFormer/dtu/scan1

In [27]:
if os.path.exists("/content/3D_object_reconstruction/MVSFormer/output/scan1"):
  shutil.rmtree('/content/3D_object_reconstruction/MVSFormer/output/scan1')

!CUDA_VISIBLE_DEVICES=0 python /content/3D_object_reconstruction/MVSFormer/test.py --batch_size 1 \
                                       --dataset test \
                                       --testpath /content/3D_object_reconstruction/MVSFormer/dtu \
                                       --testlist /content/3D_object_reconstruction/MVSFormer/lists/dtu/test.txt \
                                       --resume /content/3D_object_reconstruction/MVSFormer/pretrained_weights/mvs_best.pth \
                                       --outdir /content/3D_object_reconstruction/MVSFormer/output \
                                       --fusibile_exe_path ./fusibile/fusibile \
                                       --interval_scale 1.06 --num_view 5 \
                                       --numdepth 192 --max_h 1152 --max_w 1536 --filter_method gipuma \
                                       --disp_threshold 0.4 --num_consistent 2 --prob_threshold 0.5,0.5,0.5,0.5 \
                                       --combine_conf \
                                       --tmps 5.0,5.0,5.0,1.0

argv: ['--batch_size', '1', '--dataset', 'test', '--testpath', '/content/3D_object_reconstruction/MVSFormer/dtu', '--testlist', '/content/3D_object_reconstruction/MVSFormer/lists/dtu/test.txt', '--resume', '/content/3D_object_reconstruction/MVSFormer/pretrained_weights/mvs_best.pth', '--outdir', '/content/3D_object_reconstruction/MVSFormer/output', '--fusibile_exe_path', './fusibile/fusibile', '--interval_scale', '1.06', '--num_view', '5', '--numdepth', '192', '--max_h', '1152', '--max_w', '1536', '--filter_method', 'gipuma', '--disp_threshold', '0.1', '--num_consistent', '2', '--prob_threshold', '0.5,0.5,0.5,0.5', '--combine_conf', '--tmps', '5.0,5.0,5.0,1.0']
################################  args  ################################
model     	mvsnet                        	<class 'str'>       
device    	None                          	<class 'NoneType'>  
config    	None                          	<class 'NoneType'>  
dataset   	test                          	<class 'str'>       
testp